In [15]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [64]:
#計測地点のosm_idを指定
start_osm = 212430769
mid_osm = []
end_osm =  212430769
start_mesh = 50325672334
end_mesh = 50325662413

In [65]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [66]:
getdbname = "kddi_06_new_param"
dbname = "kddi_trkn06_monthly_anbun"
trknName = "trkn_06_grp"
sttime = "2022-04-01 06:00:00"

In [67]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        if dis_km == 0:
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値20.38369489547639距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値20.899804521151776距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値19.875809555115705距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値19.716666666666665距離0.0 km
50325662413
存在メッシュ：50325662413値21.416666666666664距離1.2342319147078133 km
50325662344
存在メッシュ：50325662344値21.050725525970382距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値20.234007116811043距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値20.53695007912503距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値20.751350133876546距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値20.034952340253763距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値21.268695717625782距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値21.203362186805293距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値20.09217768200142距離0.27262804673558266 km
50325672123
存

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値112.09971941623341距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値114.4981112061367距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値109.73954636396907距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値109.0距離0.0 km
50325662413
存在メッシュ：50325662413値116.9距離1.2342319147078133 km
50325662344
存在メッシュ：50325662344値115.19944999323492距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値111.40411150361211距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値112.81190526965948距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値113.80823493585768距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値110.47909224784593距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値116.21237029563356距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値115.90876153476187距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値110.74502177714386距離0.27262804673558266 km
50325672123
存在メッシュ：50325672123値113.53

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値107.99436441291833距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値109.27957857881586距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値106.72963033005936距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値106.33333333333333距離0.0 km
50325662413
存在メッシュ：50325662413値110.56666666666666距離1.2342319147078133 km
50325662344
存在メッシュ：50325662344値109.65540147316807距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値107.62161249349678距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値108.37599987023945距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値108.90989804579715距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値107.12592706952081距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値110.1981899896433距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値110.03549668740403距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値107.26842939112773距離0.27262804673558266 km
50325672123

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値20.38369489547639距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値20.899804521151776距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値19.875809555115705距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値19.716666666666665距離0.0 km
50325662413
存在メッシュ：50325662413値21.416666666666664距離1.2342319147078133 km
50325662344
存在メッシュ：50325662344値21.050725525970382距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値20.234007116811043距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値20.53695007912503距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値20.751350133876546距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値20.034952340253763距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値21.268695717625782距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値21.203362186805293距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値20.09217768200142距離0.27262804673558266 km
50325672123
存

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
50325672143 is kddi none!
50325672122
存在メッシュ：50325672122値114.4981112061367距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値109.73954636396907距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値109.0距離0.0 km
50325662413
50325662413 is kddi none!
50325662344
存在メッシュ：50325662344値115.19944999323492距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値111.40411150361211距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値112.81190526965948距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値113.80823493585768距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値110.47909224784593距離0.23108137431498288 km
50325662431
50325662431 is kddi none!
50325662342
存在メッシュ：50325662342値115.90876153476187距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値110.74502177714386距離0.27262804673558266 km
50325672123
50325672123 is kddi none!
50325672142
存在メッシュ：50325672142値113.13556243931565距離0.6461067275785141 km
33.8057291665:33.8

...

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値112.29113201850048距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値115.95955827942848距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値108.68116268750542距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値107.55距離0.0 km
50325662413
存在メッシュ：50325662413値119.63333333333333距離1.2342319147078133 km
50325662344
存在メッシュ：50325662344値117.03228110779601距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値111.22717476818308距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値113.38044582384623距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値114.90436778163884距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値109.81232464069261距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値118.58157903868距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値118.11719855000494距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値110.21907339331077距離0.27262804673558266 km
50325672123
存在メッシュ：503256

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値107.80449158714569距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値109.0846458547523距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値106.54473677232949距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値106.15距離0.0 km
50325662413
存在メッシュ：50325662413値110.36666666666667距離1.2342319147078133 km
50325662344
50325662344 is kddi none!
50325672321
存在メッシュ：50325672321値107.43320719496596距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値108.18462454266636距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値108.71642075690295距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値106.93947328840723距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値109.99964068522213距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値109.83758790779484距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値107.0814145772519距離0.27262804673558266 km
50325672123
存在メッシュ：50325672123値108.56943895333293距離0.70817

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値38.0176640999465距離0.4842750164752852 km
50325672122
50325672122 is kddi none!
50325672332
存在メッシュ：50325672332値37.53467509979958距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値37.38333333333333距離0.0 km
50325662413
50325662413 is kddi none!
50325662344
存在メッシュ：50325662344値38.65199715051432距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値37.87531395749024距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値38.16340677459276距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値38.36729702273881距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値37.6860167680191距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値38.85928252885328距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値38.79715162209262距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値37.7404369459556距離0.27262804673558266 km
50325672123
存在メッシュ：50325672123値38.31094431544121距離0.7081775743614449 km
50325672142
503256721

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値115.83018819206482距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値119.27091902990075距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値112.4442859229936距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値111.38333333333334距離0.0 km
50325662413
存在メッシュ：50325662413値122.71666666666668距離1.2342319147078133 km
50325662344
存在メッシュ：50325662344値120.27705906202479距離0.968551776618439 km
50325672321
存在メッシュ：50325672321値114.83226966762919距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値116.85188941638913距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値118.28122311473254距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値113.505237823914距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値121.73019367306081距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値121.29463680092421距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値113.88674010223171距離0.27262804673558266 km
50325672123
存

33.8057291665:33.8161458335:132.777343750:132.777343750
50325672143
存在メッシュ：50325672143値114.63328029459937距離0.4842750164752852 km
50325672122
存在メッシュ：50325672122値116.88999493235647距離0.858980293826149 km
50325672332
存在メッシュ：50325672332値112.41252674753207距離0.11554072466034683 km
50325672334
存在メッシュ：50325672334値111.71666666666667距離0.0 km
50325662413
存在メッシュ：50325662413値119.15距離1.2342319147078133 km
50325662344
50325662344 is kddi none!
50325672321
存在メッシュ：50325672321値113.97876314474895距離0.3755988790220571 km
50325672141
存在メッシュ：50325672141値115.30339609761208距離0.595541156918559 km
50325672124
存在メッシュ：50325672124値116.24087084681967距離0.7511996216770406 km
50325672314
存在メッシュ：50325672314値113.10838637666517距離0.23108137431498288 km
50325662431
存在メッシュ：50325662431値118.50298977184086距離1.1268022277926024 km
50325662342
存在メッシュ：50325662342値118.21731570570421距離1.0793688578871927 km
50325672323
存在メッシュ：50325672323値113.35860698862061距離0.27262804673558266 km
50325672123
存在メッシュ：50325672123値115.9817645843998距離0.7081

In [68]:
connection.close()

In [63]:
#CreateGeojsonFile